In [1]:
import boto3
import gzip
from io import BytesIO
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
s3 = boto3.resource('s3')

In [2]:
BUCKET_NAME='cryptocompare-ohlcv-hourly'
my_bucket = s3.Bucket(BUCKET_NAME)

# Get list of all files in bucket and iterate through filenames 
Base=[]
Quote=[]
Exchange=[]
File_Name=[]
for my_bucket_object in my_bucket.objects.all():
    # Download file and do something with data...
    file_name = my_bucket_object.key
    if file_name.endswith('.csv.gz'):
        File_Name.append(file_name)
        file_info_arr = file_name.split('-')
        base_currency = file_info_arr[0]
        quote_currency = file_info_arr[1]
        exchange = file_info_arr[2]
        Base.append(base_currency)
        Quote.append(quote_currency)
        Exchange.append(exchange)
#        print('Filename:{}\nBase Currency: {}\nQuoteCurrency: {}\nExchange:{}\n'
#              .format(file_name, base_currency, quote_currency, exchange))

In [321]:
#File_Name

### 1. history invested coin list
Set the intersection coin list which are both history invested and in data file as ```intersect_list```, which include 91 coins.

In [5]:
with open("97_coins_list.txt", 'r') as f:
    coins = [line.rstrip('\n') for line in f]
Base=list(set(Base))
intersect_num=len(list(set(Base)&set(coins)))
intersect_list=list(set(Base)&set(coins))
print(len(coins)-intersect_num,"coins invested in history but not in file: ",list(set(coins)-set(intersect_list)))

6 coins invested in history but not in file:  ['BKBT', 'WIN2', 'BTT2', 'IOT', 'ATOM2', 'RDN']


### 2. Exchange List
Found that we could found data file for all 91 coins, so we do not need refer other quote type. We save the name of datafile into ```Include_File```, so that we could iterate those name one by one to get data from boto3.

In [6]:
Exchange_Priority=['HuobiPro','Binance','Kucoin','BitTrex', 'Bitfinex','Kraken', 'Poloniex']
Include_File=[]
Found_List=[]
Quote=list(set(Quote))
for coin in intersect_list:
    for exchange in Exchange_Priority:
        tmp=coin+"-"+'BTC'+"-"+exchange+"-hr.csv.gz"
        if tmp in File_Name:
            Include_File.append(tmp)
            Found_List.append(coin)
            break
len(Include_File)
len(Found_List)
print("coins which are not quoted in BTC: ",set(intersect_list)-set(Found_List))

coins which are not quoted in BTC:  {'BTC', 'JST', 'BCV', 'XMX', 'USD', 'SEELE', 'USDT'}


In [322]:
#Include_File

### 3. Get data from boto3

In [8]:
# Get file from s3 as binary, unzip, and read contents
def read_s3_file(bucketname, filename):
    obj = s3.Object(bucketname, filename)
    body = obj.get()['Body'].read()
    gzipfile = BytesIO(body)
    gzipfile = gzip.GzipFile(fileobj=gzipfile)
    content = gzipfile.read()
    return content

### Set Bitcoin as standard

In [49]:
file='BTC-USDT-HuobiPro-hr.csv.gz'
DF=read_s3_file(BUCKET_NAME,file)
DF=pd.read_csv(StringIO(str(DF,'utf-8')))
new=pd.DataFrame(DF[["timestamp","close"]])
Data=new.rename(columns={"close":file.split("-")[0]})
Data1=Data
print(Data.shape[0])

for file in Include_File[:5]:
    DF=read_s3_file(BUCKET_NAME,file)
    DF=pd.read_csv(StringIO(str(DF,'utf-8')))
    new=pd.DataFrame(DF[["timestamp","close"]])
    print(file)
    new=new.rename(columns={"close":file.split("-")[0]})
    Data=pd.merge(Data,new,on="timestamp",how="left")
    print(new.shape[0],Data.shape[0])

22011
COTI-BTC-Binance-hr.csv.gz
4002 22012
AST-BTC-HuobiPro-hr.csv.gz
6003 22018
REP-BTC-Binance-hr.csv.gz
20010 22034
CTXC-BTC-HuobiPro-hr.csv.gz
6003 22050
EDO-BTC-Binance-hr.csv.gz
6003 22072
BTT-BTC-HuobiPro-hr.csv.gz
6003 22120
AERGO-BTC-Kucoin-hr.csv.gz
6003 22196
OGN-BTC-Binance-hr.csv.gz
4002 22212
DACC-BTC-Kucoin-hr.csv.gz
6003 22380
SKM-BTC-HuobiPro-hr.csv.gz
6003 22668
BTM-BTC-HuobiPro-hr.csv.gz
22011 23264
PIVX-BTC-Binance-hr.csv.gz
6003 24368
CND-BTC-Binance-hr.csv.gz
6003 26576
STORJ-BTC-HuobiPro-hr.csv.gz
22011 30840
USE-BTC-Kucoin-hr.csv.gz
4002 39033
LOKI-BTC-Kucoin-hr.csv.gz
6003 55737
GNT-BTC-HuobiPro-hr.csv.gz
22011 88841
DASH-BTC-HuobiPro-hr.csv.gz
22011 155049
BLZ-BTC-HuobiPro-hr.csv.gz
6003 287145
RLC-BTC-Binance-hr.csv.gz
6003 549929


In [222]:
file='BTC-USDT-HuobiPro-hr.csv.gz'
day=90
DF=read_s3_file(BUCKET_NAME,file)
DF=pd.read_csv(StringIO(str(DF,'utf-8')))
#new=pd.DataFrame(DF[["timestamp","close"]]).iloc[-day*24:]
#Data=new.rename(columns={"close":file.split("-")[0]})
#Data1=Data
#print(Data.shape[0])
DF.columns

Index(['timestamp', 'time', 'close', 'high', 'low', 'open', 'volumefrom',
       'volumeto'],
      dtype='object')

In [199]:
def GetDateFile(file,datatype):
    DF=read_s3_file(BUCKET_NAME,file)
    DF=pd.read_csv(StringIO(str(DF,'utf-8')))
    
    new=pd.DataFrame(DF[["timestamp",datatype]])
    Data=new.rename(columns={"close":file.split("-")[0]})
    #print(Data.shape[0])
    return Data

def ChangeStrtoDatetime(Data,from_date,to_date): #change string type date to datetime type date in datafile
    Data["timestamp"]=[datetime.datetime.strptime(dd,"%Y-%m-%d %H:%M:%S") for dd in Data["timestamp"]]
    #Data["timestamp"]=[int(datetime.datetime.timestamp(datetime.datetime.strptime(dd,"%Y-%m-%d %H:%M:%S"))) for dd in Data["timestamp"])
    Data["timestamp"]=[int(datetime.datetime.timestamp(dd)) for dd in Data["timestamp"]]
    Data=Data.loc[Data["timestamp"]>=from_date]
    Data=Data.loc[Data["timestamp"]<=to_date]
    return(Data)

### Get pre-30 days data and concat

In [123]:
import datetime

6003


In [114]:
# convert string type to datetime
to_date=Data["timestamp"].iloc[-1]
to_date
datetime_to_date= datetime.datetime.strptime(to_date,"%Y-%m-%d %H:%M:%S")
datetime_from_date=datetime_to_date+datetime.timedelta(days=-30)
from_date=int(datetime.datetime.timestamp(datetime_from_date))
to_date=int(datetime.datetime.timestamp(datetime_to_date))

1589587200

In [299]:
COLUMN=[file.split("-")[0] for file in Include_File]
#Include_File[0].split("-")[0]
COLUMN.insert(0,"BTC")
len(COLUMN)

85

In [241]:
def GetPanelData(datatype,day):
    # get datefile with whole period
    file='BTC-USDT-HuobiPro-hr.csv.gz'
    Data=GetDateFile(file,datatype)
    # get int type from_date and to_date ()
    to_date=Data["timestamp"].iloc[-1]
    datetime_to_date= datetime.datetime.strptime(to_date,"%Y-%m-%d %H:%M:%S")
    datetime_from_date=datetime_to_date+datetime.timedelta(days=-day)
    from_date=int(datetime.datetime.timestamp(datetime_from_date))
    to_date=int(datetime.datetime.timestamp(datetime_to_date))

    # change string date type to int date type
    Data=ChangeStrtoDatetime(Data,from_date,to_date)
    Data.index=Data["timestamp"]
    del Data["timestamp"]
    DF=Data
    index=DF.index
    DF.reset_index(drop=True,inplace=True)

    for file in Include_File:
        Data=GetDateFile(file,datatype)
        Data=ChangeStrtoDatetime(Data,from_date,to_date)
        Data.index=Data["timestamp"]
        del Data["timestamp"]
        Data.reset_index(drop=True, inplace=True)
        DF=pd.concat([DF, Data], axis=1)
    DF.index=[datetime.datetime.utcfromtimestamp(dd).strftime('%Y-%m-%d %H:%M:%S') for dd in index]
    return DF

In [242]:
varlist=['close', 'high', 'low', 'open', 'volumefrom',
       'volumeto']
mydict={}
for datatype in varlist:
    print(datatype)
    day=90
    vars()[datatype]=GetPanelData(datatype,day)

close
high
low
open
volumefrom
volumeto


In [300]:
for datatype in varlist:
    vars()[datatype].columns=COLUMN

In [302]:
OPEN=open.copy()
HIGH=high.copy()
LOW=low.copy()
CLOSE=close.copy()
VOLUMEFROM=volumefrom.copy()
VOLUMETO=volumeto.copy()

### Construct Features

In [303]:
# factor_1: open
factor_1=open.shift(1)

# factor_2: high
factor_2=high.shift(1)

# factor_3: low
factor_3=low.shift(1)

# factor_4: close
factor_4=close.shift(1)

# factor_5: high-low
factor_5=high.sub(low).shift(1)

In [311]:
# factor_6： # of Trade
factor_6=volumefrom.add(volumeto).div(2).shift(1)

# factor_7: US Dollar Volume

# factor_8: cryptocurrency Volume

In [ ]:
# factor_9: # of Trades

# factor_10:

# factor_11: 


In [336]:
# factor_12: returns_1
factor_12=close.diff(1).div(close.shift(1)).shift(1)

# factor_13: returns_2
factor_13=close.diff(1).div(close.shift(1)).shift(2)

# factor_14: returns_3
factor_14=close.diff(1).div(close.shift(1)).shift(3)

# factor_15: returns_4
factor_15=close.diff(1).div(close.shift(1)).shift(4)

# factor_16: returns_5
factor_16=close.diff(1).div(close.shift(1)).shift(5)

# factor_17: Moving Average(MA)
factor_17=close.diff(1).div(close.shift(1)).rolling(window=5).mean()

In [320]:
# factor_18: Correlation MA


# factor_18: Correlation Close
#close.rolling(window=1).corr()

# factor_20: sum of return_3
factor_20=close.diff(1).div(close.shift(1)).rolling(window=3).sum()

# factor_21: sum of return_5
factor_21=close.diff(1).div(close.shift(1)).rolling(window=5).sum()

# factor_22: sum of return_5-sum of return_3
factor_22=factor_21.sub(factor_20)

# factor_23: Relative Strength Index: RSI_6

# factor_24: Relative Strength Index: RSI_7


In [337]:
target=factor_12=close.diff(1).div(close.shift(1))

In [338]:
factor_1.to_excel(r'factors/factor_1.xlsx')
factor_2.to_excel(r'factors/factor_2.xlsx')

In [344]:
factor_3.to_excel(r'factors/factor_3.xlsx')
factor_4.to_excel(r'factors/factor_4.xlsx')

In [343]:
target.to_excel(r"factors/target.xlsx")

In [342]:
new=pd.read_excel(r'factors/factor_1.xlsx',index_col=0)
new

,BTC,COTI,AST,REP,CTXC,EDO,BTT,AERGO,OGN,DACC,...,TRX,BRD,CELR,PPT,ADA,ETH,ARN,MTC,LSK,R
2020-03-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-17 01:00:00,5027.01,0.000009,0.000002,0.000009,0.000007,0.000009,4.000000e-08,0.000004,0.000009,2.000000e-09,...,0.000002,0.000009,0.000009,0.000009,0.000005,0.02208,0.000009,0.000002,0.000160,0.0
2020-03-17 02:00:00,5070.25,0.000009,0.000002,0.000009,0.000007,0.000009,4.000000e-08,0.000004,0.000009,2.000000e-09,...,0.000002,0.000009,0.000009,0.000009,0.000005,0.02209,0.000009,0.000002,0.000164,0.0
2020-03-17 03:00:00,5187.90,0.000009,0.000002,0.000009,0.000007,0.000009,4.000000e-08,0.000004,0.000009,2.000000e-09,...,0.000002,0.000009,0.000009,0.000009,0.000005,0.02225,0.000009,0.000002,0.000166,0.0
2020-03-17 04:00:00,5278.43,0.000009,0.000002,0.000009,0.000008,0.000009,4.000000e-08,0.000004,0.000009,2.000000e-09,...,0.000002,0.000009,0.000009,0.000009,0.000005,0.02210,0.000009,0.000002,0.000167,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-14 20:00:00,9384.84,NaN,0.000007,NaN,0.000011,NaN,3.000000e-08,NaN,NaN,NaN,...,0.000002,NaN,NaN,NaN,0.000008,0.02502,NaN,NaN,0.000129,NaN
2020-06-14 21:00:00,9395.41,NaN,0.000007,NaN,0.000011,NaN,3.000000e-08,NaN,NaN,NaN,...,0.000002,NaN,NaN,NaN,0.000008,0.02500,NaN,NaN,0.000129,NaN
2020-06-14 22:00:00,9382.01,NaN,0.000007,NaN,0.000011,NaN,3.000000e-08,NaN,NaN,NaN,...,0.000002,NaN,NaN,NaN,0.000008,0.02495,NaN,NaN,0.000129,NaN
2020-06-14 23:00:00,9346.83,NaN,0.000007,NaN,0.000011,NaN,3.000000e-08,NaN,NaN,NaN,...,0.000002,NaN,NaN,NaN,0.000008,0.02493,NaN,NaN,0.000129,NaN


# Put into DataRobot

In [323]:
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
import datarobot as dr
import numpy as np
from datarobot.enums import AUTOPILOT_MODE
from datarobot.errors import ClientError

In [324]:
dr.Client(token='NWVkZmU1MDQ3OTI4NTYwOTJlNDRiYTRlOjV4Qyt0bVRhOWttOXZCa24yRlBvU1JhQnI4blZ3c3hUVTJQdTVYV2ZpcGc9', endpoint='https://app.datarobot.com/api/v2')

In [ ]:
url = 'https://s3.amazonaws.com/datarobot_public_datasets/10k_diabetes.xlsx'
project = dr.Project.create(url, project_name='10K Advanced Modeling')
print('Project ID: {}'.format(project.id))

In [339]:
filename='factors/factor_1.xlsx'
project_name="Feature Selection"
project = dr.Project.create(sourcedata=filename, project_name=project_name)
print('Project ID: {}'.format(project.id))

Project ID: 5f15cbfbab9af3052018cf68


In [341]:
# Increase the worker count to your maximum available the project runs faster.
project.set_worker_count(-1)

Project(Feature Selection)

In [ ]:
target_feature_name = 'readmitted'
project.set_target(target_feature_name, mode=AUTOPILOT_MODE.QUICK)